In [15]:
import numpy as np
import pandas as pd
from collections import Counter

In [2]:
data_ori = pd.read_csv("hr-analytics.csv")

In [3]:
data_ori.head()

,name,satisfaction_level,last_evaluation,number_projects,average_monthly_hours,time_spent_company,work_accident,left,promotion_last_5_years,department,salary,salary_level
0,SMITH,0.38,0.53,2,157,3,0,1,0,sales,low,1
1,JOHNSON,0.80,0.86,5,262,6,0,1,0,sales,medium,2
2,WILLIAMS,0.11,0.88,7,272,4,0,1,0,sales,medium,2
3,BROWN,0.72,0.87,5,223,5,0,1,0,sales,low,1
4,JONES,0.37,0.52,2,159,3,0,1,0,sales,low,1


In [4]:
data = data_ori.copy()

In [5]:
data_dptmt = list(set(data_ori['department']))

In [6]:
data_dptmt_dict = {}
code = 0
for dptmt in data_dptmt:
    data_dptmt_dict[dptmt] = code
    code += 1

In [7]:
data_dptmt_dict

{'IT': 8,
 'RandD': 2,
 'accounting': 3,
 'hr': 4,
 'management': 7,
 'marketing': 9,
 'product_mng': 6,
 'sales': 1,
 'support': 5,
 'technical': 0}

In [8]:
data = data_ori.drop(['salary', 'left', 'name'], axis=1, inplace=False)
data['department'] = data_ori['department'].map(data_dptmt_dict)
label = data_ori['left']
name = data_ori['name']
data_label = pd.DataFrame(name).join(label)

In [9]:
def k_shape(split_list):
    k_l = []
    for i in range(len(split_list)):
        k_l.append(len(split_list[i]))
    return k_l

In [10]:
def k_means(data_set, k, norm=2):
    m = data_set.shape[0]
    mean_vector = data_set[np.random.choice(m, k, replace=False)]
    Flag = True
    while Flag:
        C = []
        for i in range(k):
            C.append([])
        for j in range(m):
            dj = ((np.abs((data_set[j] - mean_vector)
                          ** norm).sum(axis=1)) ** (1 / norm)).argmin()
            C[dj].append(data_set[j].tolist())
        mean_prev = mean_vector.copy()
        for i in range(k):
            mean_vector[i] = np.array(C[i]).mean(axis=0).tolist()
        mean_delta = np.min(np.abs(mean_prev - mean_vector))
        if mean_delta == 0.0:
            Flag = False
    return C

In [11]:
def data_prefix(data_set):
    mean = data_set.mean(axis=1).reshape((-1, 1))
    std = data_set.var(axis=1).reshape((-1, 1))
    return (data_set - mean) / std

In [14]:
data_n = data_prefix(np.array(data))

In [12]:
def run(data):
    data_n = data_prefix(np.array(data))
    res = k_means(data_n, 2, norm=2)
    k_res = []
    for i in range(2):
        i_res = []
        for j in range(len(res[i])):
            i_res.append(np.where(data_n== np.array(res[i][j]))[0][0])
        k_res.append(i_res)
    if len(k_res[0]) > len(k_res[1]):
        pred_0 = np.array(k_res[0])
        pred_1 = np.array(k_res[1])
    else:
        pred_0 = np.array(k_res[1])
        pred_1 = np.array(k_res[0])
    real_0 = np.where(np.array(data_label['left']) == 0)[0]
    real_1 = np.where(np.array(data_label['left']) == 1)[0]
    s_r_0 = set(real_0.tolist())
    s_r_1 = set(real_1.tolist())
    p_r_0 = set(pred_0.tolist())
    p_r_1 = set(pred_1.tolist())
    pred_ok = (len(p_r_0) - len(p_r_0.difference(s_r_0))) + len(p_r_1) - (len(p_r_1.difference(s_r_1)))
    print(pred_ok / data_n.shape[0])

In [13]:
run(data)

0.480098673244883
